In [ ]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib notebook

In [ ]:
#get info from CSV created in part one of the challenge into a df.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

In [ ]:
#Check data for correct types
city_data_df.info()

In [ ]:
#get minimum and maximum temperature values from user.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to add a rain and snow values
Rain= input("Do you want it to be raining? (yes/no)?")
Snow= input("Do you want it to be snowing? (yes/no)?")

In [ ]:
#apply filter find the cities from the cities listing
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if Rain=="no" and Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0) & (preferred_cities_df["Snow inches(last 3 hrs)"]<=0)]                                                                          

elif Rain=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0),:]
                                                 
elif Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Snow inches(last 3 hrs)"]<=0),:]                                             

else:
    preferred_cities_df

preferred_cities_df.info()

In [ ]:
#get rid of the null values
preferred_cities_df=preferred_cities_df.dropna()

In [ ]:
# Print results for rain and snow
rain_count=preferred_cities_df.loc[preferred_cities_df["Rain inches(last 3 hrs)"]>0, ["Rain inches(last 3 hrs)"]].count()
snow_count=preferred_cities_df.loc[preferred_cities_df["Snow inches(last 3 hrs)"]>0, ["Snow inches(last 3 hrs)"]].count()
print(f'rain count:{rain_count},snow count:{snow_count}')

In [ ]:
# Create df to store hotel weather information and location
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

In [ ]:
# create parameters for hotel search
params = {"radius": 5000,"type": "lodging","key": g_key}

In [ ]:
#apply parameters and loop through the data
for index, row in hotel_df.iterrows():
    #return latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add location key to latitude and longitude the parameters dictionary
    params["location"] = f"{lat},{lng}"

    #set variable for base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #add base url and parameters created above and ask for a json package reture
    hotels = requests.get(base_url, params=params).json()
    #if no error, then add hotel to df
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
#check for correct data types
hotel_df.head()

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather description</dt><dd>{Current description}</dd>
</dl>
"""

In [ ]:
#add the df row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
#create heatmap of temperature for vacation spots and a city marker
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=100,point_radius=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#plot the data.
fig

In [ ]:
#create csv file variable
output_data_file = "weather_data/WeatherPy_vacation.csv"
#build data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")